In [1]:
from pylab import*
import pandas as pd
from scipy.io import arff
import plotly.express as px
import seaborn as sns

In [2]:
df1 = pd.DataFrame(arff.loadarff('1year.arff')[0])
df2 = pd.DataFrame(arff.loadarff('2year.arff')[0])
df3 = pd.DataFrame(arff.loadarff('3year.arff')[0])
df4 = pd.DataFrame(arff.loadarff('4year.arff')[0])
df5 = pd.DataFrame(arff.loadarff('5year.arff')[0])

FileNotFoundError: ignored

In [ ]:
columns=[
"net profit / total assets"
,"total liabilities / total assets"
,"working capital / total assets"
,"current assets / short-term liabilities"
,"[(cash + short-term securities + receivables - short-term liabilities) / (operating expenses - depreciation)] * 365"
,"retained earnings / total assets"
,"EBIT / total assets"
,"book value of equity / total liabilities"
,"sales / total assets"
,"equity / total assets"
,"(gross profit + extraordinary items + financial expenses) / total assets"
,"gross profit / short-term liabilities"
,"(gross profit + depreciation) / sales"
,"(gross profit + interest) / total assets"
,"(total liabilities * 365) / (gross profit + depreciation)"
,"(gross profit + depreciation) / total liabilities"
,"total assets / total liabilities"
,"gross profit / total assets"
,"gross profit / sales"
,"(inventory * 365) / sales"
,"sales (n) / sales (n-1)"
,"profit on operating activities / total assets"
,"net profit / sales"
,"gross profit (in 3 years) / total assets"
,"(equity - share capital) / total assets"
,"(net profit + depreciation) / total liabilities"
,"profit on operating activities / financial expenses"
,"working capital / fixed assets"
,"logarithm of total assets"
,"(total liabilities - cash) / sales"
,"(gross profit + interest) / sales"
,"(current liabilities * 365) / cost of products sold"
,"operating expenses / short-term liabilities"
,"operating expenses / total liabilities"
,"profit on sales / total assets"
,"total sales / total assets"
,"(current assets - inventories) / long-term liabilities"
,"constant capital / total assets"
,"profit on sales / sales"
,"(current assets - inventory - receivables) / short-term liabilities"
,"total liabilities / ((profit on operating activities + depreciation) * (12/365))"
,"profit on operating activities / sales"
,"rotation receivables + inventory turnover in days"
,"(receivables * 365) / sales"
,"net profit / inventory"
,"(current assets - inventory) / short-term liabilities"
,"(inventory * 365) / cost of products sold"
,"EBITDA (profit on operating activities - depreciation) / total assets"
,"EBITDA (profit on operating activities - depreciation) / sales"
,"current assets / total liabilities"
,"short-term liabilities / total assets"
,"(short-term liabilities * 365) / cost of products sold)"
,"equity / fixed assets"
,"constant capital / fixed assets"
,"working capital"
,"(sales - cost of products sold) / sales"
,"(current assets - inventory - short-term liabilities) / (sales - gross profit - depreciation)"
,"total costs /total sales"
,"long-term liabilities / equity"
,"sales / inventory"
,"sales / receivables"
,"(short-term liabilities *365) / sales"
,"sales / short-term liabilities"
,"sales / fixed assets","a predire"]

In [ ]:
df1.columns=columns
df2.columns=columns
df3.columns=columns
df4.columns=columns
df5.columns=columns

In [ ]:
df1.info

In [ ]:
df1.isnull().sum()

In [ ]:
df1.isnull().sum().sort_values()

In [ ]:
df1.describe()

In [ ]:
def switch(x):
  if b'0' == x:
    return 0
  else:
    return 1

In [ ]:
df1['a predire']=df1['a predire'].transform(switch)
df2['a predire']=df2['a predire'].transform(switch)
df3['a predire']=df3['a predire'].transform(switch)
df4['a predire']=df4['a predire'].transform(switch)
df5['a predire']=df5['a predire'].transform(switch)
df1['a predire']

In [ ]:
df1corr=df1.copy()
df1corr.dropna(axis=0,inplace=True)
corr = df1corr.corr()
ax = sns.heatmap(corr)
ax

In [ ]:
df1.hist(figsize=(80,60))

In [ ]:
df1['a predire'].hist()

In [ ]:
from sklearn.decomposition import PCA
# You must normalize the data before applying the fit method
pcadf1=df1.copy()
pcadf1.drop(columns='a predire',inplace=True)
pcadf1.dropna(axis=0,inplace=True)
df1_normalized=(pcadf1 - pcadf1.mean()) / pcadf1.std()
#df1_normalized.dropna(axis=1,inplace=True)
pca = PCA(n_components=pcadf1.shape[1])
pca.fit(df1_normalized)
# Reformat and view results
loadings = pd.DataFrame(pca.components_.T,
columns=['PC%s' % _ for _ in range(len(df1_normalized.columns))],
index=pcadf1.columns)
print(loadings)
plot(pca.explained_variance_ratio_)
ylabel('Explained Variance')
xlabel('Components')
grid()
show()
print(pca.explained_variance_ratio_)

In [ ]:
def pexpdata(l,p):
  for i in range(len(l)):
    if l[i]>p:
      return i

In [ ]:
comp=pca.explained_variance_ratio_
cum_ex_var_r=[comp[:i].sum() for i in range(len(comp))]
plot(cum_ex_var_r)
ylabel('cumulative Explained Variance')
xlabel('Components')
grid()
show()
print(cum_ex_var_r[44])

In [ ]:
reduc=pca.components_[:pexpdata(cum_ex_var_r,0.99)]
reddf1=df1.copy()
reddf1.dropna(axis=0,inplace=True)
apredire=reddf1['a predire']
reddf1.drop(columns='a predire',inplace=True)
reddf1=reddf1.dot(reduc.T)
reddf1['a predire']=apredire
reddf1

In [ ]:
reddf1['a predire'].value_counts(0)

In [ ]:
from sklearn.model_selection import train_test_split
X = reddf1.loc[:, reddf1.columns != "a predire"]
y = apredire
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=20)

In [ ]:
from sklearn.neighbors import NearestCentroid
clf=NearestCentroid()
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
acc=1-array([abs(list(y_test)[i]-pred[i]) for i in range(len(pred))]).sum()/len(pred)
print(acc)

In [ ]:
pcadf1=df2.copy()
pcadf1.drop(columns='a predire',inplace=True)
pcadf1.dropna(axis=0,inplace=True)
df1_normalized=(pcadf1 - pcadf1.mean()) / pcadf1.std()
#df1_normalized.dropna(axis=1,inplace=True)
pca = PCA(n_components=pcadf1.shape[1])
pca.fit(df1_normalized)
# Reformat and view results
loadings = pd.DataFrame(pca.components_.T,
columns=['PC%s' % _ for _ in range(len(df1_normalized.columns))],
index=pcadf1.columns)
comp=pca.explained_variance_ratio_
cum_ex_var_r=[comp[:i].sum() for i in range(len(comp))]
reduc=pca.components_[:pexpdata(cum_ex_var_r,0.99)]
reddf1=df2.copy()
reddf1.dropna(axis=0,inplace=True)
apredire=reddf1['a predire']
reddf1.drop(columns='a predire',inplace=True)
reddf1=reddf1.dot(reduc.T)
reddf1['a predire']=apredire
X = reddf1.loc[:, reddf1.columns != "a predire"]
y = apredire
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=20)
clf=NearestCentroid()
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
acc=1-array([abs(list(y_test)[i]-pred[i]) for i in range(len(pred))]).sum()/len(pred)
print(acc)

In [ ]:
pcadf1=df3.copy()
pcadf1.drop(columns='a predire',inplace=True)
pcadf1.dropna(axis=0,inplace=True)
df1_normalized=(pcadf1 - pcadf1.mean()) / pcadf1.std()
#df1_normalized.dropna(axis=1,inplace=True)
pca = PCA(n_components=pcadf1.shape[1])
pca.fit(df1_normalized)
# Reformat and view results
loadings = pd.DataFrame(pca.components_.T,
columns=['PC%s' % _ for _ in range(len(df1_normalized.columns))],
index=pcadf1.columns)
comp=pca.explained_variance_ratio_
cum_ex_var_r=[comp[:i].sum() for i in range(len(comp))]
reduc=pca.components_[:pexpdata(cum_ex_var_r,0.99)]
reddf1=df3.copy()
reddf1.dropna(axis=0,inplace=True)
apredire=reddf1['a predire']
reddf1.drop(columns='a predire',inplace=True)
reddf1=reddf1.dot(reduc.T)
reddf1['a predire']=apredire
X = reddf1.loc[:, reddf1.columns != "a predire"]
y = apredire
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=20)
clf=NearestCentroid()
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
acc=1-array([abs(list(y_test)[i]-pred[i]) for i in range(len(pred))]).sum()/len(pred)
print(acc)

In [ ]:
pcadf1=df4.copy()
pcadf1.drop(columns='a predire',inplace=True)
pcadf1.dropna(axis=0,inplace=True)
df1_normalized=(pcadf1 - pcadf1.mean()) / pcadf1.std()
#df1_normalized.dropna(axis=1,inplace=True)
pca = PCA(n_components=pcadf1.shape[1])
pca.fit(df1_normalized)
# Reformat and view results
loadings = pd.DataFrame(pca.components_.T,
columns=['PC%s' % _ for _ in range(len(df1_normalized.columns))],
index=pcadf1.columns)
comp=pca.explained_variance_ratio_
cum_ex_var_r=[comp[:i].sum() for i in range(len(comp))]
reduc=pca.components_[:pexpdata(cum_ex_var_r,0.99)]
reddf1=df4.copy()
reddf1.dropna(axis=0,inplace=True)
apredire=reddf1['a predire']
reddf1.drop(columns='a predire',inplace=True)
reddf1=reddf1.dot(reduc.T)
reddf1['a predire']=apredire
X = reddf1.loc[:, reddf1.columns != "a predire"]
y = apredire
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=20)
clf=NearestCentroid()
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
acc=1-array([abs(list(y_test)[i]-pred[i]) for i in range(len(pred))]).sum()/len(pred)
print(acc)

In [ ]:
pcadf1=df5.copy()
pcadf1.drop(columns='a predire',inplace=True)
pcadf1.dropna(axis=0,inplace=True)
df1_normalized=(pcadf1 - pcadf1.mean()) / pcadf1.std()
#df1_normalized.dropna(axis=1,inplace=True)
pca = PCA(n_components=pcadf1.shape[1])
pca.fit(df1_normalized)
# Reformat and view results
loadings = pd.DataFrame(pca.components_.T,
columns=['PC%s' % _ for _ in range(len(df1_normalized.columns))],
index=pcadf1.columns)
comp=pca.explained_variance_ratio_
cum_ex_var_r=[comp[:i].sum() for i in range(len(comp))]
reduc=pca.components_[:pexpdata(cum_ex_var_r,0.99)]
reddf1=df5.copy()
reddf1.dropna(axis=0,inplace=True)
apredire=reddf1['a predire']
reddf1.drop(columns='a predire',inplace=True)
reddf1=reddf1.dot(reduc.T)
reddf1['a predire']=apredire
X = reddf1.loc[:, reddf1.columns != "a predire"]
y = apredire
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=20)
clf=NearestCentroid()
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
acc=1-array([abs(list(y_test)[i]-pred[i]) for i in range(len(pred))]).sum()/len(pred)
print(acc)